In [5]:
import pandas as pd
import numpy as np
DTST = '~/Data/DZDT/data/2_words/words.csv'

In [6]:
data = pd.read_csv(DTST, sep='\t')

data.head()

,word,AR,BER,DZ,EN,FR
0,كريس,1,0,1,0,0
1,جونسون,1,0,1,1,0
2,(,1,1,1,1,1
3,لاعب,1,0,1,0,0
4,كرة,1,0,1,1,0


In [7]:
def to_bin(vec):
    base = 1
    res = 0
    for e in vec:
        res += base * e
        base = base * 2
    return res

data.iloc[1, 1:].tolist(), to_bin(data.iloc[1, 1:].tolist())

([1, 0, 1, 1, 0], 13)

In [8]:
data['hash'] = list(map(lambda l: to_bin(l), data.iloc[:, 1:].values.tolist()))

data.head()

,word,AR,BER,DZ,EN,FR,hash
0,كريس,1,0,1,0,0,5
1,جونسون,1,0,1,1,0,13
2,(,1,1,1,1,1,31
3,لاعب,1,0,1,0,0,5
4,كرة,1,0,1,1,0,13


In [10]:
# data['hash'].value_counts()
hash = data['hash'].unique()
hash

array([ 5, 13, 31, 29,  1, 21, 25, 23,  9,  7, 15, 27, 17,  3, 19, 11, 28,
       20, 24, 30, 16, 22, 26, 18,  8, 12, 14, 10,  4,  6,  2])

In [11]:
# stats
counts = data['hash'].value_counts()

counts

hash
4     3808202
1     2304728
8     1104555
16     975697
5      529010
24     352139
2      147471
25     133527
9       97626
20      64710
29      55052
28      42079
17      29494
12      29139
13      10427
31       9244
21       6475
6        5816
27       2766
30       2461
26       1988
18       1834
22       1293
10       1008
14        694
3         541
11        441
7         309
23        308
15        220
19        199
Name: count, dtype: int64

In [37]:
d1 = data.iloc[[0, 1, 3], :-1]
d1

,word,FR,AR,KAB,DZ,EN
0,Éthique,1,0,0,1,1
1,de,1,1,1,1,1
3,sollicitude,1,0,0,1,0


In [38]:
d2 = data.iloc[[4, 2], :-1]
d2

,word,FR,AR,KAB,DZ,EN
4,L,1,1,1,1,1
2,la,1,1,1,1,1


In [39]:
d3 = pd.concat([d1, d2], axis=0)
d3

,word,FR,AR,KAB,DZ,EN
0,Éthique,1,0,0,1,1
1,de,1,1,1,1,1
3,sollicitude,1,0,0,1,0
4,L,1,1,1,1,1
2,la,1,1,1,1,1


In [61]:
MACROBATCHSIZE = 10 

macrobatches = []
first = True

for val in data['hash'].unique():
    didx = np.argwhere((data['hash'] == val).to_numpy()).reshape(-1)
    nbr = len(didx)
    step = int(nbr/MACROBATCHSIZE)
    e = MACROBATCHSIZE * step
    for i in range(MACROBATCHSIZE):
        if first:
            macrobatches.append(data.iloc[didx[:step], :-1])
        else:
            s, e = step*i, step*(i+1)
            macrobatches[i] = pd.concat([macrobatches[i], data.iloc[didx[s:e], :-1]], axis=0)
    print(step, nbr, e)
    if e < nbr:
        macrobatches[0] = pd.concat([macrobatches[0], data.iloc[didx[e:], :-1]], axis=0)
    
    first = False   


4316 43167 43160
776 7765 7760
6934 69344 69340
5565 55659 55650
166 1666 1660
14330 143301 143300
150 1508 1500
71 718 710
640 6408 6400
131220 1312202 1312200
38382 383824 383820
272 2725 2720
4097 40973 40970
180 1809 1800
19 195 190
18 185 180
274097 2740971 2740970
54049 540494 540490
1055 10555 10550
10 108 100
20 208 200
8924 89249 89240
31 313 310
20 205 200
5735 57352 57350
245 2457 2450
24 249 240
69 695 690
428355 4283556 4283550
3209 32092 32090
164799 1647991 1647990


In [63]:
total_size = len(data)
cumulative_size = 0
for i in range(MACROBATCHSIZE):
    print(len(macrobatches[i]))
    cumulative_size += len(macrobatches[i])

total_size, cumulative_size

1147942
1147778
1147778
1147778
1147778
1147778
1147778
1147778
1147778
1147778


(11477944, 11477944)

In [64]:
for i in range(MACROBATCHSIZE):
    macrobatches[i].to_csv(DTST + str(i), index=False)